In [1]:
import gurobipy as gp
from gurobipy import GRB

In [2]:
#feasability check for relaxations does not work yet... 

# def check_nonconvex_feasibility(model, relaxation_solution):
#     constraints = model.getConstrs()
#     for constr in constraints:
#         lhs = sum(model.getCoeff(constr, var) * relaxation_solution[var.index]
#                   for var in model.getVars())
#         if lhs < constr.RHS - 1e-6:  # Replace with appropriate tolerance
#             print(f"Relaxation solution violates constraint {constr.ConstrName}")
#             return False
#     print("Relaxation solution is feasible for the relaxed problem.")
#     return True


In [22]:
def my_callback(model, where):
    if where == GRB.Callback.MIPSOL:
        # Feasible integer solution found
        try:
            solution = model.cbGetSolution(model.getVars())
            print("Feasible solution found:", solution)
            model._feasible_solutions.append(solution)
        except gp.GurobiError as e:
            print(f"Error retrieving feasible solution: {e}")
    
    elif where == GRB.Callback.MIPNODE:
        try:
            # Check node status
            node_status = model.cbGet(GRB.Callback.MIPNODE_STATUS)
            
            if node_status == GRB.Status.OPTIMAL:
                # Optimal relaxation available
                relaxation_solution = model.cbGetNodeRel(model.getVars())
                #is_feasible = test_relaxation_feasibility(model, relaxation_solution)
                # print(f"Optimal relaxation solution is {is_feasible} and the solution is: {relaxation_solution}")
                # model._relaxation_solutions.append({"status": "Optimal",  "feasiable": is_feasible, "solution": relaxation_solution})
                
                print(f"Optimal relaxation solution is: {relaxation_solution}")
                model._relaxation_solutions.append({"status": "Optimal", "solution": relaxation_solution})
            
            elif node_status == GRB.Status.INFEASIBLE:
                # Node relaxation is infeasible
                print("Infeasible relaxation at node.")
                model._relaxation_solutions.append({"status": "Infeasible", "solution": None})
            
            elif node_status == GRB.Status.UNBOUNDED:
                # Node relaxation is unbounded
                print("Unbounded relaxation at node.")
                model._relaxation_solutions.append({"status": "Unbounded", "solution": None})
            
            elif node_status == GRB.Status.INF_OR_UNBD:
                # Node relaxation is infeasible or unbounded
                print("Infeasible or unbounded relaxation at node.")
                model._relaxation_solutions.append({"status": "InfOrUnbd", "solution": None})
            
            else:
                print(f"Unhandled node status: {node_status}")
                model._relaxation_solutions.append({"status": "Unknown", "solution": None})
        
        except gp.GurobiError as e:
            print(f"Error retrieving relaxation solution: {e}")

# Read the problem
number = "0031"
model = gp.read(f"QPLIB_{number}.lp")

# Solution storage
model._feasible_solutions = []
model._relaxation_solutions = []

#model.setParam("NodeLimit", 100)  # Explore a limited number of nodes

# Optimize
model.optimize(my_callback)

Read LP format model from file QPLIB_0031.lp
Reading time = 0.00 seconds
obj: 32 rows, 60 columns, 120 nonzeros
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 22.1.0 22A8380)

CPU model: Apple M2 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 32 rows, 60 columns and 120 nonzeros
Model fingerprint: 0x00d24133
Model has 464 quadratic objective terms
Variable types: 30 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [3e+01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Feasible solution found: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,

In [23]:
# Print solutions
print("\nCollected feasible solutions:")
for idx, sol in enumerate(model._feasible_solutions):
    print(f"Solution {idx + 1}: {sol}")


Collected feasible solutions:
Solution 1: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
Solution 2: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Solution 3: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Solution 4: [0.0, 0.0, 0.0, 0.

In [24]:
print("\nCollected relaxation solutions:")
for idx, sol in enumerate(model._relaxation_solutions):
    print(f"Relaxation {idx + 1} ({sol['status']}): {sol['solution']}")


Collected relaxation solutions:
Relaxation 1 (Optimal): [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0]
Relaxation 2 (Optimal): [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07676254776178146, 0.08279614432825083, 0.07713007083010298, 0.0, 0.0, 0.7633112370798647, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, -0.0, 1.0, 0.08279614432825083, 0.07713007083010298, 1.0, 1.0, 0.7633112370798647, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0]
Relaxation 3 (Optimal): [0.0, 0.0, 0.0, 0.0, 0.0, 0.6321310852742655, 0.0, 0.0, 0.0, 0.1794945242316262, 0.18837439049410823, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [27]:
import re
import numpy as np

# Initialize variables
objective_section = False
constraints_section = False
binary_section = False

objective_lines = []
constraint_lines = []
binary_lines = []

# Read the file
with open(f'QPLIB_{number}.txt', 'r') as f:
    for line in f:
        line = line.strip()
        if line.startswith('Minimize'):
            objective_section = True
            constraints_section = False
            binary_section = False
            continue
        elif line.startswith('Subject To'):
            objective_section = False
            constraints_section = True
            binary_section = False
            continue
        elif line.startswith('Binary'):
            objective_section = False
            constraints_section = False
            binary_section = True
            continue
        elif line.startswith('End'):
            objective_section = False
            constraints_section = False
            binary_section = False
            continue

        if objective_section:
            objective_lines.append(line)
        elif constraints_section:
            constraint_lines.append(line)
        elif binary_section:
            binary_lines.append(line)

# Collect variables
variables_set = set()

# From the 'Binary' section
binary_text = ' '.join(binary_lines)
binary_vars = re.findall(r'\b[b,x]\d+\b', binary_text)
variables_set.update(binary_vars)

# From the 'Subject To' section
constraint_text = ' '.join(constraint_lines)
vars_in_constraints = re.findall(r'\b[b,x]\d+\b', constraint_text)
variables_set.update(vars_in_constraints)

# From the objective function
objective_text = ' '.join(objective_lines)
vars_in_objective = re.findall(r'\b[b,x]\d+\b', objective_text)
variables_set.update(vars_in_objective)

# Function to extract variable type and number
def extract_var_num(var_name):
    m = re.match(r'([bx])(\d+)', var_name)
    if m:
        var_type = m.group(1)
        var_num = int(m.group(2))
        return var_type, var_num
    else:
        return None, None

# Create list of variables with type and number
variables_info = []
for var in variables_set:
    var_type, var_num = extract_var_num(var)
    if var_type is not None:
        variables_info.append((var_type, var_num, var))

# Sort variables: 'x' variables first, then 'b' variables, both sorted by number
variables_info.sort(key=lambda x: (0 if x[0]=='x' else 1, x[1]))

# Create variable_indices mapping
variable_indices = {var_info[2]: idx for idx, var_info in enumerate(variables_info)}

n = len(variable_indices)

# Now, parse the objective function
# Extract the quadratic terms inside the square brackets
quadratic_match = re.search(r'\[(.*)\]/2', objective_text)
if quadratic_match:
    quadratic_text = quadratic_match.group(1)
else:
    print("No quadratic terms found in the objective function.")
    quadratic_text = ''

# Replace '-' with '+ -' to split terms correctly
quadratic_text = quadratic_text.replace('-', '+ -')
terms = quadratic_text.split('+')

Q = np.zeros((n, n))

# Parse quadratic terms and build Q matrix
for term in terms:
    term = term.strip()
    if term == '':
        continue
    # Match terms like '52.8828 x2^2' or '-63.7552 x2 * x3'
    # First, split the term to separate the coefficient
    match = re.match(r'([+-]?\s*\d*\.?\d+(?:[eE][+-]?\d+)?)\s*(.*)', term)
    if match:
        coeff_str = match.group(1).replace(' ', '')
        coeff = float(coeff_str)
        vars_part = match.group(2)
        vars_part = vars_part.strip()
        # Check for squared terms
        squared_match = re.match(r'([bx]\d+)\^2', vars_part)
        if squared_match:
            var_name = squared_match.group(1)
            if var_name in variable_indices:
                i = variable_indices[var_name]
                # Update Q[i, i]
                Q[i, i] += coeff
            else:
                print(f"Variable {var_name} not found.")
        else:
            # Check for cross terms
            cross_match = re.match(r'([bx]\d+)\s*\*\s*([bx]\d+)', vars_part)
            if not cross_match:
                # Try matching 'x2*x3' without spaces
                cross_match = re.match(r'([bx]\d+)\*(?:\s*)([bx]\d+)', vars_part)
            if cross_match:
                var1_name = cross_match.group(1)
                var2_name = cross_match.group(2)
                if var1_name in variable_indices and var2_name in variable_indices:
                    i = variable_indices[var1_name]
                    j = variable_indices[var2_name]
                    # Update Q[i, j] and Q[j, i]
                    Q[i, j] += coeff
                    Q[j, i] += coeff
                else:
                    print(f"Variables {var1_name} or {var2_name} not found.")
            else:
                print(f"Could not parse term: {term}")
    else:
        print(f"Could not parse term: {term}")

# Now process the constraints (same as before)
# ... [Code for processing constraints remains the same] ...

# Output the variable indices mapping
print("Variable Indices:")
for var, idx in variable_indices.items():
    print(f"{var}: {idx}")

# Output the results
print("\nMatrix A (for Ax ≤ b):")
print(A)
print("\nVector b:")
print(b_vector)
print("\nMatrix E (for Ex = d):")
print(E)
print("\nVector d:")
print(d)
print("\nCost matrix Q:")
print(Q)
print("\nBinary variable indices:")
print(binary_indices)


Variable Indices:
x2: 0
x3: 1
x4: 2
x5: 3
x6: 4
x7: 5
x8: 6
x9: 7
x10: 8
x11: 9
x12: 10
x13: 11
x14: 12
x15: 13
x16: 14
x17: 15
x18: 16
x19: 17
x20: 18
x21: 19
x22: 20
x23: 21
x24: 22
x25: 23
x26: 24
x27: 25
x28: 26
x29: 27
x30: 28
x31: 29
b32: 30
b33: 31
b34: 32
b35: 33
b36: 34
b37: 35
b38: 36
b39: 37
b40: 38
b41: 39
b42: 40
b43: 41
b44: 42
b45: 43
b46: 44
b47: 45
b48: 46
b49: 47
b50: 48
b51: 49
b52: 50
b53: 51
b54: 52
b55: 53
b56: 54
b57: 55
b58: 56
b59: 57
b60: 58
b61: 59

Matrix A (for Ax ≤ b):
[[ 0.  0.  0. ...  0.  0.  0.]
 [-1.  0.  0. ...  0.  0.  0.]
 [ 0. -1.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]

Vector b:
[5. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]

Matrix E (for Ex = d):
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 2. 1. 1. 2. 2.

In [31]:
var_info = [i[0] for i in variables_info]

In [32]:
var_info

['x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b']

In [ ]:
# I want to use the bipartite graph representation of the constraint matrix A for the linear constraints Ax≤b
# To parameterize a graph neural network to predict the feasibility of a solution (x,x_bin) where x are continuous variables and x_bin are binary
# The cost function for the prediction should be ReLU(Ax-b) 

In [ ]:
# + ReLU(Ex-d) + ReLU(d-Ex)